In [35]:
# --------------
# Import the Data
# --------------
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import locale
from locale import atof


# Get the current working directory & define tha data path
print(os.getcwd())
path = str(os.getcwd())+'/data/data.csv'

# Define the data frame
df = pd.read_csv(path, sep=';', encoding='ISO-8859-1', header=10, skipinitialspace=True)
print(df.info())

/Users/tp/Documents/Dev/private/TU/Challenge 2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34722 entries, 0 to 34721
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Betreiber               34722 non-null  object
 1   Straße                  34722 non-null  object
 2   Hausnummer              34722 non-null  object
 3   Adresszusatz            4846 non-null   object
 4   Postleitzahl            34722 non-null  int64 
 5   Ort                     34722 non-null  object
 6   Bundesland              34722 non-null  object
 7   Kreis/kreisfreie Stadt  34722 non-null  object
 8   Breitengrad             34722 non-null  object
 9   Längengrad              34722 non-null  object
 10  Inbetriebnahmedatum     34722 non-null  object
 11  Anschlussleistung       34722 non-null  object
 12  Normalladeeinrichtung   34722 non-null  object
 13  Anzahl Ladepunkte       34722 non-null  int64 
 14  Stecker

In [36]:
# --------------
# Milestone 2 - Basic Data infos
# --------------

# Log the shape (rows, columns) of the df [would work with: print(len(df.columns)) as well]
print(df.shape)

# Log the column types
print(df.dtypes)



(34722, 26)
Betreiber                 object
Straße                    object
Hausnummer                object
Adresszusatz              object
Postleitzahl               int64
Ort                       object
Bundesland                object
Kreis/kreisfreie Stadt    object
Breitengrad               object
Längengrad                object
Inbetriebnahmedatum       object
Anschlussleistung         object
Normalladeeinrichtung     object
Anzahl Ladepunkte          int64
Steckertypen1             object
P1 [kW]                   object
Public Key1               object
Steckertypen2             object
P2 [kW]                   object
Public Key2               object
Steckertypen3             object
P3 [kW]                   object
Public Key3               object
Steckertypen4             object
P4 [kW]                   object
Public Key4               object
dtype: object


In [37]:
# --------------
# Milestone 3 - Add proper types
# --------------


def preprocess_lat_long(data, column):

  data[column] = data[column].apply(lambda x: x.replace(',','.'))
  data[column] = data[column].apply(lambda x: x.replace(',','.'))

  return data[column]

preprocess_lat_long(df, 'Breitengrad')
preprocess_lat_long(df, 'Längengrad')

def preprocess_rstrip(data, column): # remove very last dot in a value in 'Breitengrad' column

  data[column] = data[column].apply(lambda x: x.rstrip('.'))

preprocess_rstrip(df, 'Breitengrad')

# Converting normal charging & fast charging to 0 & 1 respectively
df['Normalladeeinrichtung'].replace(['Normalladeeinrichtung', 'Schnellladeeinrichtung'],
                        [0, 1], inplace=True)

# Set locale to work with german commas as decimal separator to convert strings to float
locale.setlocale(locale.LC_NUMERIC, '')

# Declare final dtypes we want to have
df_dtypes = {
    "Betreiber"                 : "string",
    "Straße"                    : "string",
    "Hausnummer"                : "string", # String, since we have stuff like 10c as well here
    "Adresszusatz"              : "string",
    "Postleitzahl"              : "uint32",
    "Ort"                       : "string",
    "Bundesland"                : "string",
    "Kreis/kreisfreie Stadt"    : "string",
    "Breitengrad"               : "float32", 
    "Längengrad"                : "float32", 
    "Inbetriebnahmedatum"       : "datetime64",
    "Anschlussleistung"         : "float32", 
    "Normalladeeinrichtung"     : "object", # Should be boolean? --> should be binary (0,1) int
    "Anzahl Ladepunkte"         : "uint32",
    "Steckertypen1"             : "string", # enum?
    "P1 [kW]"                   : "float32", 
    "Public Key1"               : "string", 
    "Steckertypen2"             : "string",
    "P2 [kW]"                   : "float32", 
    "Public Key2"               : "string", 
    "Steckertypen3"             : "string",
    "P3 [kW]"                   : "float32", 
    "Public Key3"               : "string", 
    "Steckertypen4"             : "string",
    "P4 [kW]"                   : "float32", 
    "Public Key4"               : "string", 
}


# Parse df types

def clean_dtypes(df, column_name, final_dtype):
    try:
        if final_dtype == "object":
            return df[column_name].astype(final_dtype)
        elif final_dtype == "uint32" or final_dtype == "uint64":
            return df[column_name].replace(np.nan, 0).astype(final_dtype)
        elif final_dtype == "float32" or final_dtype == "float64":
            return df[column_name].astype(str).apply(atof).astype(final_dtype)
        elif final_dtype == "string":
            return df[column_name].astype(final_dtype)
        elif final_dtype == "datetime64":
          return pd.to_datetime(df[column_name], format="%d.%m.%Y")
        return df[column_name].astype(final_dtype) # return the passed dtype without modification as a fallback
    except Exception as e:
        print("Error while converting column: " + column_name + " to dtype: " + final_dtype)
        print(e)
        return df[column_name]
    

def convert_dtypes(df, dtypes):
    for column, dtype in dtypes.items():
        df[column] = clean_dtypes(df, column, dtype)
    return df

converted_df = convert_dtypes(df, df_dtypes)
print(converted_df.dtypes)

0        4.857853e+08
1        4.852898e+07
2        4.855354e+07
3        4.817996e+07
4        4.848039e+07
             ...     
34717    5.093467e+06
34718    5.093467e+06
34719    5.106077e+06
34720    5.106080e+09
34721    5.103204e+09
Name: Breitengrad, Length: 34722, dtype: float64
Betreiber                         string
Straße                            string
Hausnummer                        string
Adresszusatz                      string
Postleitzahl                      uint32
Ort                               string
Bundesland                        string
Kreis/kreisfreie Stadt            string
Breitengrad                      float32
Längengrad                       float32
Inbetriebnahmedatum       datetime64[ns]
Anschlussleistung                float32
Normalladeeinrichtung             object
Anzahl Ladepunkte                 uint32
Steckertypen1                     string
P1 [kW]                          float32
Public Key1                       string
Steckertypen

<bound method DataFrame.info of                                                Betreiber        Straße  \
0                           EnBW mobility+ AG und Co.KG    Hauptstraße   
1                                       SWU Energie GmbH  Lindenstraße   
2                                       SWU Energie GmbH    Öllengasse   
3                                       SWU Energie GmbH    Sterngasse   
4                           EnBW mobility+ AG und Co.KG         Heuweg   
...                                                  ...           ...   
34717                                         Aral Pulse     Aralallee   
34718                                         Aral Pulse     Aralallee   
34719                  bft Tankstelle Christian Hollmotz      Dorfstr.   
34720                               Thüringer Energie AG   Dorfstraße    
34721  IGW Ingenieurgesellschaft für Wasserkraftanlag...   Hauptstraße   

      Hausnummer Adresszusatz  Postleitzahl            Ort         Bundesland  

In [38]:
# --------------
# 3.2 - Fix missing values
# --------------

# Remove Adresszusatz column
converted_df = converted_df.drop(columns=['Adresszusatz'])

# Set columns P1-P4 to 0 if NaN
converted_df['P1 [kW]'] = converted_df['P1 [kW]'].fillna(0)
converted_df['P2 [kW]'] = converted_df['P2 [kW]'].fillna(0)
converted_df['P3 [kW]'] = converted_df['P3 [kW]'].fillna(0)
converted_df['P4 [kW]'] = converted_df['P4 [kW]'].fillna(0)

# Set columns Public Key1-Public Key4 to '-' if NaN
converted_df['Public Key1'] = converted_df['Public Key1'].fillna('-')
converted_df['Public Key2'] = converted_df['Public Key2'].fillna('-')
converted_df['Public Key3'] = converted_df['Public Key3'].fillna('-')
converted_df['Public Key4'] = converted_df['Public Key4'].fillna('-')

# Set columns Steckertypen1-Steckertypen4 to '-' if NaN
converted_df['Steckertypen1'] = converted_df['Steckertypen1'].fillna('-')
converted_df['Steckertypen2'] = converted_df['Steckertypen2'].fillna('-')
converted_df['Steckertypen3'] = converted_df['Steckertypen3'].fillna('-')
converted_df['Steckertypen4'] = converted_df['Steckertypen4'].fillna('-')



# Check for missing values
df_null_values = converted_df.isnull().sum()
print('NANs?', df_null_values)


NANs? Betreiber                 0
Straße                    0
Hausnummer                0
Postleitzahl              0
Ort                       0
Bundesland                0
Kreis/kreisfreie Stadt    0
Breitengrad               0
Längengrad                0
Inbetriebnahmedatum       0
Anschlussleistung         0
Normalladeeinrichtung     0
Anzahl Ladepunkte         0
Steckertypen1             0
P1 [kW]                   0
Public Key1               0
Steckertypen2             0
P2 [kW]                   0
Public Key2               0
Steckertypen3             0
P3 [kW]                   0
Public Key3               0
Steckertypen4             0
P4 [kW]                   0
Public Key4               0
dtype: int64


In [39]:
# --------------
# 3.3 - Outlier check
# --------------

# entries with more outlets than stated in "Anzahl Ladepunkte"

# House numbers with 0 and above 1000

# entries with more than 4 outlets

# Postleitzahlen with 0 and above 10000

In [40]:
# --------------
# Milestone 3.3 - Draw missing values
# --------------
# Draw missing values into a diagram
def draw_missing_values(df):
    plt.figure(figsize=(15,5))
    sns.heatmap(df.isnull(), cbar=False, yticklabels=False, cmap='Greys')
    plt.xticks(rotation=45, fontsize=6)
    plt.tight_layout()
    plt.savefig('pyplots/milestone_three_missing_values.pdf')
    plt.close()
    
draw_missing_values(converted_df)

In [41]:
# Drop all rows with NaN.
df = df.dropna(axis=0)
df_null_values = df.isnull().sum()
print('NANs_After_Update?', df_null_values)
print('// complete ........ Pre-Processing')

NANs_After_Update? Betreiber                 0
Straße                    0
Hausnummer                0
Adresszusatz              0
Postleitzahl              0
Ort                       0
Bundesland                0
Kreis/kreisfreie Stadt    0
Breitengrad               0
Längengrad                0
Inbetriebnahmedatum       0
Anschlussleistung         0
Normalladeeinrichtung     0
Anzahl Ladepunkte         0
Steckertypen1             0
P1 [kW]                   0
Public Key1               0
Steckertypen2             0
P2 [kW]                   0
Public Key2               0
Steckertypen3             0
P3 [kW]                   0
Public Key3               0
Steckertypen4             0
P4 [kW]                   0
Public Key4               0
dtype: int64
// complete ........ Pre-Processing
